In [1]:
import sys
import os
import pandas as pd
sys.path.append('../../../py_pkg')
import platform_designer as qpd
import pd_ip.pd_ip as pd_ip
from pd_ip.custom_ips import niosv_aes
import pd_ip.quartus_ips as quartus_ips
from analysis import tunable_tdc_sweeper as tts
import time
import shutil
# import paramiko
# from scp import SCPClient

KeyboardInterrupt: 

In [ ]:
# import Quartus Platform Designer system as instance objects
sopcinfo_header = os.path.abspath("../../../../EXPERIMENTS/quartus_projects/niosv_aes/qsys_headers/hps_0.h")
pd_0 = qpd.pd_system("u0", [sopcinfo_header], 0xC0000000, "/dev/mem")
# initialize all modules and setup MMIO
pd_0.initialize(True)

In [ ]:
# setup DMA *TODO* convert to python module - currently a C driver
dma = os.open("/dev/msgdma_tdc", os.O_RDONLY)
pd_0.pd_ip_insts["DMA_to_SDRAM"].dma_fh=dma

In [ ]:
# Connect reset PIOs to proper instances
pd_0.pd_ip_insts["theta_pll_reconfig"].reset_pio = pd_0.pd_ip_insts["theta_pll_reset_pio"]
pd_0.pd_ip_insts["phi_pll_reconfig"].reset_pio = pd_0.pd_ip_insts["phi_pll_reset_pio"]
pd_0.pd_ip_insts["pulsegenerator"].reset_pio = pd_0.pd_ip_insts["tdc_reset_pio"]
# Connect locked PIOs to PLLs
pd_0.pd_ip_insts["theta_pll_reconfig"].locked_pio = pd_0.pd_ip_insts["theta_locked"]
pd_0.pd_ip_insts["phi_pll_reconfig"].locked_pio = pd_0.pd_ip_insts["phi_locked"]
# Configure PLL number of outputs
pd_0.pd_ip_insts["theta_pll_reconfig"].num_out = 2
pd_0.pd_ip_insts["phi_pll_reconfig"].num_out = 1

In [ ]:
# Import PLL configs for sweep
theta_pll_configs = "../pll_configs/theta.csv"
theta_cfg_params = pd.read_csv(theta_pll_configs)
phi_pll_configs   = "../pll_configs/phi.csv"
phi_cfg_params = pd.read_csv(phi_pll_configs)

In [ ]:
# Hold pulsegenerator and TDC in reset until PLL is configured
pd_0.pd_ip_insts["pulsegenerator"].hold_reset()

# Hold PLL theta in reset until after phi is reset
pd_0.pd_ip_insts["theta_pll_reconfig"].hold_reset()

# # Initialize phi clock - Mostly to ensure bandwidth setting is correct
pd_0.pd_ip_insts["phi_pll_reconfig"].update_all_50(64, 2, [32], "low", 1, [{"phase_updn":0, "phase_amt":0}])

# # Hold PLL theta in reset until after phi is reset
pd_0.pd_ip_insts["theta_pll_reconfig"].reset_pll()
pd_0.pd_ip_insts["pulsegenerator"].reset()

In [ ]:
# AES has no direct interface, so create an instance manually
niosv_aes_ip_dict = {
    "COMPONENT_NAME" : "niosv_aes",
    "COMPONENT_TYPE" : "niosv_aes",
    "BASE" : 0x000,
    "SPAN" : 0x000,
    "END"  : 0x000
}
pd_0.pd_ip_insts["niosv_aes"] = niosv_aes.niosv_aes(
    ip_dict=niosv_aes_ip_dict, mmio_init=False, periph_mem=pd_0.pd_ip_insts["peripheral_mem"])

# All AES traffic flows through IP-Sync instance, so copy it's MMIO interface
pd_0.pd_ip_insts["ip_sync_niosv"].copy_mmio_ifc(pd_0.pd_ip_insts["niosv_aes"])

# Check AES decrypt
pd_0.pd_ip_insts["niosv_aes"].pre()
pd_0.pd_ip_insts["niosv_aes"].run()
pd_0.pd_ip_insts["niosv_aes"].post()

# Hold SEC Works AES in reset during Theta sweep
pd_0.pd_ip_insts["niosv_aes"].hold_reset()

In [ ]:
# Construct a tdc_sweeper instance
tdc_sweeper = tts.TunableTDCSweeper(
    pd_0.pd_ip_insts["theta_pll_reconfig"], 
    pd_0.pd_ip_insts["phi_pll_reconfig"], 
    pd_0.pd_ip_insts["pulsegenerator"], 
    pd_0.pd_ip_insts["DMA_to_SDRAM"], 
    tdc_len = 128,
    target_ip=pd_0.pd_ip_insts["niosv_aes"])

print(tdc_sweeper.target_ip.print_ip_info())

In [ ]:
# Sweep theta, collect plot results
# Specify theta sweepe parameters
theta_sweep_params = {
    "delay_lower_bound" : 2200,
    "delay_upper_bound" : 4500,
    "min var prop dist" : 28,
    "max var prop dist" : 36,
    "target rising mu"  : 32
}
time_now = time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime())
out_fh = f"../../../archived_results/data/niosv_theta_sweep_{time_now}.csv"

# Start theta sweep
tdc_sweeper.sweep_theta(
    theta_cfg_params=theta_cfg_params, 
    theta_samples=2**12, 
    sweep_params=theta_sweep_params, 
    tune_param="mid", 
    sweep_data_fh=out_fh, 
    debug=0)

In [ ]:
# Plot theta sweep results
tdc_sweeper.plot_theta_sweep_results(out_fh)

In [9]:
# Specify fast theta tune parameters
theta_sweep_params = {
    "delay_lower_bound" : 2211 + 1075,
    "delay_upper_bound" : 2211 + 1150,
    "target rising mu"  : 64
}

# Start fast theta sweep
tdc_sweeper.pop_tune_theta(
    theta_cfg_params=theta_cfg_params,
    theta_samples=2**12, 
    sweep_params=theta_sweep_params,
    tune_rise=False,
    tuning_param="mid")

print(f"\tRise pop: {tdc_sweeper.tuned_theta_avg_rise}")
print(f"\tFall pop: {tdc_sweeper.tuned_theta_avg_fall}")

100% (374 of 374) |######################| Elapsed Time: 0:00:02 Time:  0:00:02


Theta tuning complete! mid falling=57.5655517578125
	          M:44
	          N:2
	          C:11
	Phase Bumps:675
	 Delay (ps):3295.4545454545528
	Rise pop: 51.27008056640625
	Fall pop: 57.5655517578125


In [11]:
# def createSSHClient(server, port, user, keyfile):
#     client = paramiko.SSHClient()
#     client.load_system_host_keys()
#     client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
#     client.connect(server, port, username=user, key_filename=keyfile)
#     return client

# ssh = createSSHClient(server, port, user, password)
# scp = SCPClient(ssh.get_transport())
# dest_dir="/home/tyler/ResearchData/phi_sweep_db/"
theta_samples = 2**12
n_packed_runs = 100
time_now = time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime())
phi_data_dir = f'../../../archived_results/data/phi_sweep_{time_now}'
os.mkdir(phi_data_dir)
tdc_sweeper.sweep_phi(phi_cfg_params, \
                      theta_samples, \
                      "bg", \
                      perpetual=False, \
                      process=False, \
                      out_dir=phi_data_dir+"/phi_bg", \
                      n_packed_runs=n_packed_runs)
tdc_sweeper.sweep_phi(phi_cfg_params, \
                      theta_samples, \
                      "target", \
                      perpetual=True, \
                      process=False, \
                      out_dir=phi_data_dir+"/phi_target", \
                      n_packed_runs=n_packed_runs)

100% (6625 of 6625) |####################| Elapsed Time: 0:03:40 Time:  0:03:40
  0% (0 of 6625) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--


UnboundLocalError: local variable 'phi_sweep_plot_data' referenced before assignment